In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=6

In [ ]:
# import torch
import jax
import jax.numpy as jnp
import flax
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
from IPython.display import display, HTML
from functools import partial
import json
from entmax_jax.activations import sparsemax, entmax15
from meta_expl.explainers import load_explainer, create_explainer
from meta_expl.models import load_model, create_model
from meta_expl.data.cifar100 import dataloader, load_data

### define args

In [ ]:
# arguments
arch = 'vit-base'
setup = 'static_teacher'  # "no_teacher", "static_teacher", "learnable_teacher"
seed = 9
batch_size = 16
max_len = 32*32
num_classes = 100
task_type = "classification"
modality = "image"

teacher_dir = 'data/cifar100-vit-models/teacher_dir'
teacher_expl_dir = 'data/cifar100-vit-models/teacher_expl_dir'
student_dir = 'data/cifar100-vit-models/student_dir'
student_expl_dir = 'data/cifar100-vit-models/student_expl_dir'

In [ ]:
from transformers import ViTFeatureExtractor
tokenizer = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
vocab_size = None

In [ ]:
# create dummy inputs for model instantiation
dummy_inputs = {"pixel_values": jnp.ones((batch_size, 3, 224, 224))}

### load models and explainers

In [ ]:
%env XLA_FLAGS=--xla_gpu_strict_conv_algorithm_picker=false
%env XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/local/cuda/

In [ ]:
teacher, teacher_params, dummy_state = load_model(teacher_dir, dummy_inputs, batch_size, max_len)
teacher_explainer, teacher_explainer_params = load_explainer(teacher_expl_dir, dummy_inputs, state=dummy_state)
student, student_params, dummy_state = load_model(student_dir, dummy_inputs, batch_size, max_len)
student_explainer, student_explainer_params = load_explainer(student_expl_dir, dummy_inputs, state=dummy_state)

In [ ]:
from meta_expl.utils import PRNGSequence
keyseq = PRNGSequence(seed)

teacher_explainer_params_non_trained={
    'normalize_head_coeffs': 'sparsemax',
    'normalizer_fn': 'softmax',
    'aggregator_idx': 'mean',
    'aggregator_dim': 'row',
    'init_fn': 'uniform',
    'layer_idx': None,
    'head_idx': None
}

# teacher, teacher_params, dummy_state = create_model(
#     key=next(keyseq),
#     inputs=dummy_inputs,
#     vocab_size=1,
#     num_classes=num_classes,
#     arch=arch,
#     batch_size=batch_size,
#     max_len=max_len,
#     embeddings=None,
# )
teacher_explainer_non_trained, teacher_explainer_params_non_trained = create_explainer(
    key=next(keyseq),
    inputs=dummy_inputs,
    state=dummy_state,
    explainer_type='attention_explainer',
    explainer_args=teacher_explainer_params_non_trained,
)

### look at the coefficients

In [ ]:
sparsemax(teacher_explainer_params_non_trained['params']['head_coeffs']).reshape(12, 12)

In [ ]:
sparsemax(teacher_explainer_params['params']['head_coeffs']).reshape(12, 12)

In [ ]:
hc = sparsemax(teacher_explainer_params['params']['head_coeffs']).reshape(12, 12)
for a, b in zip(*hc.nonzero()):
    print(a, b, hc[a,b])

In [ ]:
coeffs = np.asarray(hc)
fig, ax = plt.subplots(figsize=(4, 4))
ax.imshow(coeffs, cmap='Greens')
ax.set_xticks(list(range(12)))
ax.set_yticks(list(range(12)))
ax.set_xlabel('Head')
ax.set_ylabel('Layer')
ax.set_title('Head coefficients')

### load data

In [ ]:
train_data = load_data(setup, "train")
valid_data = load_data(setup, "valid")
test_data = load_data(setup, "test")
labels = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'cra', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm',]


In [ ]:
labels = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'cra', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm',]

## plot some samples

In [ ]:
test_data[0]

In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(16, 12))
for i in range(25):
    ax = axs[i//5, i%5]
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    sample = test_data[i+100]
    ax.imshow(sample['img'])
    ax.set_title('{}: {}'.format(i, labels[sample['fine_label']]))

## plot explanations

In [ ]:
from captum.attr import visualization

In [ ]:
def get_expl(data, idx=0, non_trained=False):
    x, y = next(dataloader([data[idx]], tokenizer=tokenizer, batch_size=1, max_len=None, shuffle=False))
    y_teacher, teacher_attn = teacher.apply(teacher_params, **x, deterministic=True)
    if non_trained:
        teacher_expl, _ = teacher_explainer_non_trained.apply(teacher_explainer_params_non_trained, x, teacher_attn)
    else:
        teacher_expl, _ = teacher_explainer.apply(teacher_explainer_params, x, teacher_attn)
    y_student, student_attn = student.apply(student_params, **x)
    student_expl, _ = student_explainer.apply(student_explainer_params, x, student_attn)
    teacher_rep = teacher_attn['hidden_states'][0][0]
    teacher_attn = np.asarray(jnp.stack(teacher_attn['attentions']).transpose([1, 0, 2, 3, 4]))
    student_attn = np.asarray(jnp.stack(student_attn['attentions']).transpose([1, 0, 2, 3, 4]))
    return {
        'teacher_attn': teacher_attn[0],
        'teacher_expl': teacher_expl.tolist()[0],
        'teacher_rep': teacher_rep.tolist()[1:],
        'student_attn': student_attn[0],
        'student_expl': student_expl.tolist()[0],
        'y_teacher': np.argmax(y_teacher.tolist()[0]),
        'y_student': np.argmax(y_student.tolist()[0]),
        'x': x['pixel_values'][0].transpose([1, 2, 0]),
        'y': y[0],
        'x_img': data[idx]['img'],
    }


In [ ]:
def show_expl(data, idx=0, non_trained=False):
    d = get_expl(data, idx, non_trained=non_trained)
    # x = np.asarray(d['x'])
    x = np.asarray(d['x_img'].resize((224, 224)))
    expl = np.asarray(d['teacher_expl'])
    # remove cls token
    e = expl[1:].reshape(14, 14)
    # repeat interleave for each patch
    e = np.repeat(np.repeat(e, 16, axis=0), 16, axis=1)
    # add dummy dim
    e = e[:, :, None]
    
    print('y_teacher:', labels[d['y_teacher']])
    print('y_student:', labels[d['y_student']])
    
    fig, axs = plt.subplots(1, 3, figsize=(6, 3))
    axs[0].set_title('original image')
    axs[0].get_xaxis().set_visible(False)
    axs[0].get_yaxis().set_visible(False)
    [axs[0].spines[a].set_linewidth(1) for a in ['top','bottom','left','right']]
    axs[0].imshow(x)
    
    
    axs[1].set_title('raw explanation')
    axs[1].get_xaxis().set_visible(False)
    axs[1].get_yaxis().set_visible(False)
    [axs[1].spines[a].set_linewidth(1) for a in ['top','bottom','left','right']]
    axs[1].imshow(e)    
    
    
    axs[2].set_title('blended explanation')
    axs[2].get_xaxis().set_visible(False)
    axs[2].get_yaxis().set_visible(False)
    [axs[2].spines[a].set_linewidth(1) for a in ['top','bottom','left','right']]
    axs[2].imshow(x, cmap='gray', interpolation='nearest',)
    axs[2].imshow(e, cmap='viridis', interpolation='bilinear', alpha=.5)
    
    return d

In [ ]:
d = show_expl(test_data, idx=114)
plt.savefig('example-butterfly-learned-explainer.pdf', bbox_inches='tight', dpi=300)

In [ ]:
d = show_expl(test_data, idx=114, non_trained=True)
plt.savefig('example-butterfly-non-trained-explainer.pdf', bbox_inches='tight', dpi=300)

In [ ]:
d = show_expl(test_data, idx=5)
plt.savefig('example-television-learned-explainer.pdf', bbox_inches='tight', dpi=300)

In [ ]:
d = show_expl(test_data, idx=5, non_trained=True)
plt.savefig('example-television-non-trained-explainer.pdf', bbox_inches='tight', dpi=300)

In [ ]:
d = show_expl(test_data, idx=2)

## plot patches 16x16 patches

In [ ]:
# Set the gridding interval: here we use the major tick interval
import matplotlib.ticker as plticker
loc = plticker.MultipleLocator(base=14)

# Add the grid
fig, ax = plt.subplots(figsize=(2, 2))
ax.xaxis.set_major_locator(loc)
ax.yaxis.set_major_locator(loc)
ax.grid(which='major', axis='both', linestyle='-')
# ax.get_xaxis().set_visible(False)
# ax.get_yaxis().set_visible(False)
ax.imshow(d['x_img'].resize((244, 244)))

## plot attention maps

In [ ]:
row_id = 0
heads_expl = d['teacher_attn']

fig, axs = plt.subplots(12, 12, figsize=(16, 16))

for l in range(12):
    for i in range(12):
        ax = axs[l, i]
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax.imshow(heads_expl[l, i, row_id, 1:].reshape(14, 14), cmap='viridis')